<a href="https://colab.research.google.com/github/Rama389/AI-Projects/blob/main/Unsupervised_Learning_Recommendation_System.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

# Load datasets
all_orders = pd.read_csv("all_except_last_orders.csv")
last_orders_subset = pd.read_csv("last_orders_subset.csv")

# Step 1: Frequency-based user-item matrix
user_item_matrix = all_orders.groupby(['Member', 'SKU']).size().unstack(fill_value=0)

# Step 2: Item-item similarity using cosine similarity
item_similarity = cosine_similarity(user_item_matrix.T)
item_similarity_df = pd.DataFrame(
    item_similarity, index=user_item_matrix.columns, columns=user_item_matrix.columns
)

# Step 3: Get popular SKUs (most frequently ordered items)
popular_skus = all_orders['SKU'].value_counts().index.tolist()

# Step 4: Generate recommendations
recommendations = []

grouped = last_orders_subset.groupby('Order')

for order_id, group in grouped:
    member = group['Member'].iloc[0]
    cart_skus = group['SKU'].tolist()

    # Collaborative Filtering Part
    similar_items = pd.Series(dtype=float)
    for sku in cart_skus:
        if sku in item_similarity_df:
            similar = item_similarity_df[sku].drop(labels=cart_skus, errors='ignore')
            similar_items = similar_items.add(similar, fill_value=0)

    # Sort based on similarity scores
    cf_top = similar_items.sort_values(ascending=False)

    # Combine CF + Popular fallback
    recommended = list(cf_top.index[:5])
    if len(recommended) < 5:
        fallback = [sku for sku in popular_skus if sku not in cart_skus and sku not in recommended]
        recommended += fallback[:5 - len(recommended)]

    # Save the final top 5 for this order
    for sku in recommended[:5]:
        recommendations.append({
            'Member': member,
            'Order': order_id,
            'SKU': sku
        })

# Convert to DataFrame and save
rec_df = pd.DataFrame(recommendations)
rec_df.to_csv("GR1_hybrid_rec_5_sets.csv", index=False)

print("✅ Hybrid recommendation file saved as GR1_hybrid_rec_5_sets.csv")

✅ Hybrid recommendation file saved as GR1_hybrid_rec_5_sets.csv
